In [1]:
import mysql.connector.pooling as pooling
from mysql.connector import Error 

# Task 1

In [2]:
dbconfig = {
    'user':'root',
    'password':'rootpass',
    'database': 'little_lemon_db'
}

try:
    pool = pooling.MySQLConnectionPool(
        pool_name='pool_b',
        pool_size=2,
        **dbconfig
    )
    print('Connection established.')
    print('Pool name:', pool.pool_name)
    print('Pool size:', pool.pool_size)
except pooling.Error as e:
    print('Unable to establish connection.')
    print(e.msg)

Connection established.
Pool name: pool_b
Pool size: 2


# Task 2

In [3]:
guests = [
    {
        'table_number': guest[0],
        'first_name': guest[1],
        'last_name': guest[2],
        'booking_time': guest[3],
        'employee_id': guest[4]
    } 
    for guest in 
    [
        [8, 'Anees', 'Java', '18:00:00', 6],
        [5, 'Bald', 'Vin', '19:00:00', 6],
        [12, 'Jay', 'Kon', '19:30:00', 6]
    ]
]

In [4]:
connections = []

for index, guest in enumerate(guests):
    insert_q = f"""INSERT INTO booking (table_no, guest_first_name, guest_last_name, booking_slot, employee_id) VALUES (
        {guest["table_number"]}, 
        '{guest["first_name"]}', 
        '{guest["last_name"]}', 
        '{guest["booking_time"]}', 
        {guest["employee_id"]}
    )"""

    try:
        guest_connection = pool.get_connection()
        connections.append(guest_connection)        
        cursor = guest_connection.cursor()
        cursor.execute(insert_q)
        guest_connection.commit()
    except Exception as e:
        print(f'Guest - {guest["first_name"]} {guest["last_name"]} - unable to connect.')
        print('Please try again later.')
        print(e)

Guest - Jay Kon - unable to connect.
Please try again later.
Failed getting connection; pool exhausted


In [5]:
for connection in connections:
    try:
        connection.close()
        print('Connection closed.')
    except Error as e:
        print('Connection unable to return to pool')
        print('Error message:', e.msg)

Connection closed.
Connection closed.


# Task 3

In [6]:
manager_query = """
SELECT employee_id, name, role
FROM employee
WHERE role LIKE '%Manager%'
"""

highest_salary_query = """
SELECT employee_id, name, annual_salary
FROM employee
WHERE annual_salary = (SELECT MAX(annual_salary) FROM employee)
"""

between_18_to_20_query = """
SELECT
    count(booking_id) num_of_booking
FROM booking
WHERE HOUR(booking_slot) BETWEEN 18 AND 20
"""

guest_waiting_query = """
SELECT 
    booking_id,
    CONCAT(guest_first_name, ' ', guest_last_name) guest
FROM booking
JOIN employee USING(employee_id)
WHERE role = 'Receptionist'
ORDER BY booking_slot
"""

In [7]:
query_list = [manager_query, highest_salary_query, between_18_to_20_query, guest_waiting_query]

for index, q in enumerate(query_list):
    if index == 0: print('Little Lemon Mangers')
    elif index == 1: print('Highest Annual Salary')
    elif index == 2: print('Bookings Between 18:00 - 20:00')
    elif index == 3: print('Waiting list')
    
    connection = pool.get_connection()
    cursor = connection.cursor()

    cursor.execute(q)
    columns = cursor.column_names
    dataset = cursor.fetchall()

    for index, col in enumerate(columns): 
        print(col, end=': ')
        for set in dataset:
            print(set[index], end=', ')
        print('')

    print('')

    connection.close()
    

Little Lemon Mangers
employee_id: 1, 2, 
name: Mario Gollini, Adrian Gollini, 
role: Manager, Assistant Manager, 

Highest Annual Salary
employee_id: 1, 
name: Mario Gollini, 
annual_salary: $70,000, 

Bookings Between 18:00 - 20:00
num_of_booking: 14, 

Waiting list
booking_id: 7, 9, 11, 13, 15, 8, 10, 12, 14, 16, 
guest: Anees Java, Anees Java, Anees Java, Anees Java, Anees Java, Bald Vin, Bald Vin, Bald Vin, Bald Vin, Bald Vin, 



# Task 4

In [8]:
basic_sales_report_procedure = """
CREATE PROCEDURE IF NOT EXISTS basic_sales_report()
BEGIN
SELECT 
	SUM(bill_amount) total_sales,
    ROUND(AVG(bill_amount), 1) average_sale,
    MIN(bill_amount) min_bill_paid,
    MAX(bill_amount) max_bill_paid
FROM booking_order;
END
"""

In [9]:
connection = pool.get_connection()
cursor = connection.cursor()

cursor.execute(basic_sales_report_procedure)
connection.commit()

cursor.callproc('basic_sales_report')

columns = next(cursor.stored_results()).column_names
results = next(cursor.stored_results())
dataset = results.fetchall()

for index, col in enumerate(columns):      
    print(col, end=': ')
    for set in dataset:
        print('$', set[index], sep='')

connection.close()

total_sales: $243
average_sale: $48.6
min_bill_paid: $37
max_bill_paid: $86


# Task 5

In [10]:
connection = pool.get_connection()
cursor = connection.cursor(buffered = True)

upcoming_query = """
SELECT 
	TIME_FORMAT(booking_slot, '%H:%i:%s') booking_slot,
    CONCAT(guest_first_name, ' ', guest_last_name) guest,
    CONCAT('assign to: ', employee.name, ' ', '[', employee.role, ']') assignment
FROM booking
JOIN employee USING(employee_id)
ORDER BY booking_slot
LIMIT 3
"""

cursor.execute(upcoming_query)
columns = cursor.column_names
dataset = cursor.fetchall()

for index, col in enumerate(columns):      
    print(col)
    for i, set in enumerate(dataset):
        print(f'{i + 1}.', set[index])
    print('')

connection.close()

booking_slot
1. 15:00:00
2. 17:30:00
3. 18:00:00

guest
1. Vanessa McCarthy
2. Marcos Romero
3. Anees Java

assignment
1. assign to: Giorgos Dioudis [Head Chef]
2. assign to: Fatma Kaya [Assistant Chef]
3. assign to: John Millar [Receptionist]

